### 建立提取链

In [1]:
! pip install --upgrade langchain-core

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [2]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] ="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" #getpass.getpass()

In [41]:
from typing import  Optional
from pydantic import BaseModel,Field

class Person(BaseModel):
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

In [38]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field

prompt_template = ChatPromptTemplate.from_messages(
    [
         
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    
    ]
)

In [28]:
prompt_template.invoke({"text": "wq is a good man"})

ChatPromptValue(messages=[SystemMessage(content='你是一个专业的提取算法。只能从文本中提取相关信息。 如果你不知道被要求提取的属性值， 则返回该属性为空。', additional_kwargs={}, response_metadata={}), HumanMessage(content='wq is a good man', additional_kwargs={}, response_metadata={})])

In [52]:
from langchain.chat_models import init_chat_model
llm = init_chat_model(
    model="THUDM/glm-4-9b-chat",
    model_provider="openai",
    api_key="sk-pvdcqtmuphqjgepywixzapshexhxcugkhnrtjnuhdotlvgtx", 
    base_url="https://api.siliconflow.cn/v1",
    temperature=0.1,
    max_retries=3,
    
)

In [54]:
structured_llm = llm.with_structured_output(schema=Person)

In [55]:
text = "Alan Smith is 6 feet tall and has blond hair."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

ValidationError: 1 validation error for Person
  Invalid JSON: invalid number at line 2 column 2 [type=json_invalid, input_value='\n-.1\n  \t\n \t\n \t\n ', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/json_invalid